# Extract and visualise the GPS data

## Extract the locations 
Extract the location with a primitive sed/aws-script.

In [ ]:
%%bash
sed 's/[<>]/ /g' /home/jovyan/work/data/garmin/*.tcx | \
    awk '
    BEGIN {print "timestamp,lat,lon"} 
    /Time / {timestamp=$2} 
    /LatitudeDegrees / {lat=$2 }
    # Assuming Longitude always comes after Latitude...
    /LongitudeDegrees / {lon=$2; print timestamp","lat","lon}' | gzip > /home/jovyan/work/data/locations.csv.gz

## Load the plotting libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime

from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import WMTSTileSource

import datashader as ds

from datashader import transfer_functions as tf
from datashader.colors import Greys9

from datashader.bokeh_ext import InteractiveImage
from functools import partial
from datashader.utils import export_image
from datashader.colors import colormap_select, Greys9, Hot, viridis, inferno
from IPython.core.display import HTML, display
from functools import partial

output_notebook()

## TODO: Find a set of coordinates which do not distort the map
EUROPE = x_range, y_range = ((-1113194.9079327357, 3896182.1777645755), 
                             (3503549.8435043739, 11068715.65937949))
plot_width  = int(800)
plot_height = int(plot_width//1.2)

def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

def base_plot(tools='pan,wheel_zoom,zoom_in,zoom_out,reset',
              plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

## Load the data

In [ ]:
locations = pd.read_csv('/home/jovyan/work/data/locations.csv.gz', parse_dates=['timestamp'])
locations

### Convert the coordinates to Web mercator format

In [ ]:
wgs84_to_web_mercator(locations)

## Plot the data


In [ ]:
tiles = {'osm': 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png',
         'carto': 'http://a.basemaps.cartocdn.com/light_all/{Z}/{X}/{Y}.png',
         'carto_dark': 'http://a.basemaps.cartocdn.com/dark_all/{Z}/{X}/{Y}.png'}

def create_image(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h,
                    x_range=x_range, y_range=y_range)
    agg = cvs.points(locations, 'x', 'y',  ds.count())
    img = tf.shade(agg, cmap=Hot, how='eq_hist')
    return tf.dynspread(img, threshold=0.3, max_px=4)

p = base_plot()
p.add_tile(WMTSTileSource(url=tiles['carto_dark']))
InteractiveImage(p, create_image)